In [ ]:
import pandas as pd
import datetime
import yfinance as yf

### 경제지표 데이터 전처리

filled_economic_indicators.csv 생성

#### yfinance에서 데이터 불러오기

In [ ]:
# 시작일과 종료일 설정
start_date = '2015-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# 데이터를 저장할 빈 데이터프레임 생성
df_final = pd.DataFrame()

# 각 지표의 티커 심볼 정의
tickers = {
    'KOSPI': '^KS11',  # KOSPI
    'USD/KRW': 'KRW=X',  # 원달러 환율
    'WTI': 'CL=F',  # WTI 원유 선물
    'VIX': '^VIX',  # VIX 지수
    'Gold': 'GC=F',  # 금 선물
    'Silver': 'SI=F',  # 은 선물
    'MOVE' : '^MOVE' # MOVE Index
}

# 각 티커에 대해 데이터 다운로드
for name, ticker in tickers.items():
    try:
        df = yf.download(ticker, start=start_date, end=end_date)
        # 종가만 사용
        df_final[name] = df['Close']
    except Exception as e:
        print(f"Error downloading {name}: {e}")

# MOVE 인덱스는 별도로 처리해야 할 수 있음 (Bloomberg 등의 유료 데이터 소스 필요)
print("Note: MOVE Index data needs to be obtained from a separate source like Bloomberg")

# 결측치 처리
df_final = df_final.fillna(method='ffill')

# 데이터 확인
print("\nFirst few rows of the data:")
print(df_final.head())

# 기본 통계 확인
print("\nBasic statistics:")
print(df_final.describe())

# CSV 파일로 저장
output_filename = '_economic_indicators_.csv'
df_final.to_csv(output_filename)
print(f"\nData saved to {output_filename}")

#### 경제지표 데이터 결측치 채우기 

In [ ]:
# 데이터 로드
df = pd.read_csv('_economic_indicators_.csv', encoding='utf-8')

# Date 컬럼을 datetime으로 변환
df['Date'] = pd.to_datetime(df['Date'])

# 시작일과 마지막일 추출
start_date = df['Date'].min()
end_date = df['Date'].max()

# 모든 날짜가 포함된 데이터프레임 생성
all_dates = pd.DataFrame(
    {'Date': pd.date_range(start_date, end_date, freq='D')}
)

# 기존 데이터와 병합
filled_df = pd.merge(all_dates, df, on='Date', how='left')

# 결측치를 이전 값으로 채우기
filled_df = filled_df.ffill()

# 결과 저장
filled_df.to_csv('filled_economic_indicators.csv', index=False)

# 결과 확인
print("처리된 데이터 샘플:")
print(filled_df.head())
print("\n결측치 확인:")
print(filled_df.isnull().sum())

### 트렌드 데이터 어종별로 그룹화

merged_trends.csv 생성  
nst_@@@ --> 그룹화_nst_@@@


#### 함수 생성

In [ ]:
import pandas as pd

def process_age_groups(file_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # 연령대 그룹 매핑 딕셔너리 생성
    age_mapping = {
        '19_24': '20대',
        '25_29': '20대',
        '30_34': '30대',
        '35_39': '30대',
        '40_44': '40대',
        '45_49': '40대',
        '50_54': '50대',
        '55_59': '50대',
        '60_80': '60대 이상'
    }
    
    # 원하는 연령대만 필터링
    df = df[df['age'].isin(age_mapping.keys())]
    
    # 새로운 연령대 컬럼 생성
    df['age_group'] = df['age'].map(age_mapping)
    
    # 일자별, 새로운 연령대별로 score 합산
    result = df.groupby(['date', 'name', 'age_group'])['score'].sum().reset_index()
    
    # 피벗 테이블로 변환하여 보기 좋게 정리
    pivot_result = result.pivot(index=['date', 'name'], 
                              columns='age_group', 
                              values='score').reset_index()
    
    # 컬럼 순서 정리
    column_order = [ 'date', 'name', '20대', '30대', '40대', '50대', '60대 이상']
    pivot_result = pivot_result[column_order]
    
    return pivot_result



#### 파일 저장

In [ ]:
# 📌 수산물 종류별 파일 경로 및 이름 설정
fish_files = {
    '광어': '../../data/raw/nst_광어_trend_2025-01-17.csv',
    '농어': '../../data/raw/nst_농어_trend_2025-01-17.csv',
    '대게': '../../data/raw/nst_대게_trend_2025-01-17.csv',
    '방어': '../../data/raw/nst_방어_trend_2025-01-17.csv',
    '연어': '../../data/raw/nst_연어_trend_2025-01-17.csv',
    '우럭': '../../data/raw/nst_우럭_trend_2025-01-17.csv',
    '참돔': '../../data/raw/nst_참돔_trend_2025-01-17.csv'
}

# 📌 개별 파일 저장 + 결과 리스트로 출력 준비
all_results = {}

# 📌 모든 파일 처리
for fish, file_path in fish_files.items():
    try:
        processed_df = process_age_groups(file_path, fish)

        # 개별 파일 저장
        output_filename = f'그룹화_nst_{fish}_trend_2025-01-17.csv'
        processed_df.to_csv(output_filename, index=False)

        # 결과 리스트에 추가 (첫 3개 행만 저장)
        all_results[fish] = processed_df.head(3)

        print(f"✅ '{output_filename}' 저장 완료!")

    except Exception as e:
        print(f"❌ Error processing {fish}: {e}")

# 📌 최종 처리된 결과 출력 (각각 3개 행만 출력)
print("\n📊 최종 처리된 결과 미리보기 (각각 상위 3개 행)\n")
for fish, df_sample in all_results.items():
    print(f"📌 {fish} 데이터 샘플:")
    print(df_sample)
    print("-" * 50)  # 구분선 추가

### 기상 데이터 전처리

weatherdata_processed.csv 생성

In [1]:
# 함수 정의

def create_weather_columns(df, station_cols, output_path):
   result_df = df.copy()
   result_df['일시'] = pd.to_datetime(result_df['일시'])
   
   final_df = pd.DataFrame({'일시': result_df['일시'].unique()})
   
   for station, columns in station_cols.items():
       station_data = result_df[result_df['지점'] == station].copy()
       
       for orig_col, new_col in columns:
           station_col = station_data[['일시', orig_col]].copy()
           final_df = pd.merge(final_df, station_col.rename(columns={orig_col: new_col}), 
                             on='일시', how='left')
   
   # 칼럼을 가나다순으로 정렬 ('일시' 컬럼은 첫번째로 유지)
   sorted_cols = ['일시'] + sorted([col for col in final_df.columns if col != '일시'])
   result = final_df[sorted_cols].sort_values('일시')
   
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result

In [ ]:
# CSV 파일 읽기
df = pd.read_csv('forecast_agg.csv', encoding='utf-8')

# 피쳐 선택
station_columns = {
	22105: [
		('기온', '광어_기온_22105'),
		('기온', '농어_기온_22105'),
		('기온', '대게_기온_22105'),
		('파주기', '대게_파주기_22105'),
		('파주기', '방어_파주기_22105'),
		('기온', '연어_기온_22105'),
		('습도', '연어_습도_22105')
	],
    
	22107: [
		('수온', '광어_수온_22107'),
		('수온', '농어_수온_22107'),
		('수온', '방어_수온_22107'),
		('수온', '연어_수온_22107'),
		('수온', '참돔_수온_22107')
	],

	22186: [
		('습도', '광어_습도_22186'),
		('습도', '농어_습도_22186'),
		('기온', '우럭_기온_22186'),
		('수온', '우럭_수온_22186')
		],
        
	22188: [
		('수온', '대게_수온_22188'),
		('습도', '대게_습도_22188')
		],        

	22189: [
		('파주기', '우럭_파주기_22189')
		],       

	22190: [
		('파주기', '광어_파주기_22190'),
        ('파주기', '농어_파주기_22190'),
        ('기온', '방어_기온_22190'),
        ('습도', '방어_습도_22190'),
        ('파주기', '연어_파주기_22190'),
        ('습도', '우럭_습도_22190'),
        ('기온', '참돔_기온_22190'),
        ('습도', '참돔_습도_22190'),
        ('파주기', '참돔_파주기_22190')
		]  

	}

result = create_weather_columns(df, station_columns, 'weatherdata_processed.csv')

### 변수 하나로 합치기

merged_all_data.csv 생성

In [ ]:
def merge_fish_trends(file_paths, output_path):
   # 첫 번째 파일 로드 및 name 컬럼 제거
   merged_df = pd.read_csv(file_paths[0], encoding='utf-8')
   merged_df = merged_df.drop('name', axis=1)
   merged_df['date'] = pd.to_datetime(merged_df['date'])
   
   # 나머지 파일들 병합
   for file_path in file_paths[1:]:
       df = pd.read_csv(file_path, encoding='utf-8')
       df = df.drop('name', axis=1)
       df['date'] = pd.to_datetime(df['date'])
       merged_df = pd.merge(merged_df, df, on='date', how='outer')
   
   # 날짜순 정렬
   result = merged_df.sort_values('date')
   
   # CSV 저장
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result


In [ ]:
# 사용 예시:
files = [
    '그룹화_nst_광어_trend_2025-01-17.csv', 
    '그룹화_nst_농어_trend_2025-01-17.csv', 
    '그룹화_nst_대게_trend_2025-01-17.csv', 
    '그룹화_nst_방어_trend_2025-01-17.csv', 
    '그룹화_nst_연어_trend_2025-01-17.csv', 
    '그룹화_nst_우럭_trend_2025-01-17.csv', 
    '그룹화_nst_참돔_trend_2025-01-17.csv'
    ]

result = merge_fish_trends(files, 'merged_trends.csv')

In [ ]:
def merge_data(file_paths, output_path):
   # 첫 번째 파일 로드 및 name 컬럼 제거
   merged_df = pd.read_csv(file_paths[0], encoding='utf-8')
   merged_df = merged_df.drop('name', axis=1)
   merged_df['date'] = pd.to_datetime(merged_df['date'])
   
   # 나머지 파일들 병합
   for file_path in file_paths[1:]:
       df = pd.read_csv(file_path, encoding='utf-8')
       df = df.drop('name', axis=1)
       df['date'] = pd.to_datetime(df['date'])
       merged_df = pd.merge(merged_df, df, on='date', how='outer')
   
   # 날짜순 정렬
   result = merged_df.sort_values('date')
   
   # CSV 저장
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result

In [ ]:
# 필요없는 칼럼 제거

df_광어 = pd.read_csv('item_price_lag_filled.csv')
df_광어 = df_광어.drop('minPrice', axis=1)
df_광어.to_csv('item_price_lag_filled.csv', index=False)

In [ ]:
def merge_all_data(output_path='merged_all_data.csv'):
   # 각 파일 로드 및 전처리
   
   economic = pd.read_csv('_filled_economic_indicators.csv', encoding='utf-8')
   economic['Date'] = pd.to_datetime(economic['Date'])
   economic = economic.rename(columns={'Date': '날짜'})
   
   trends = pd.read_csv('merged_trends.csv', encoding='utf-8')
   trends['date'] = pd.to_datetime(trends['date'])
   trends = trends.rename(columns={'date': '날짜'})
   
   weather = pd.read_csv('_weatherdata_processed.csv', encoding='utf-8')
   weather['date'] = pd.to_datetime(weather['date'])
   weather = weather.rename(columns={'date': '날짜'})
   
   # 모든 날짜 추출
   all_dates = pd.concat([
        economic['날짜'], 
        trends['날짜'], 
       weather['날짜']
   ]).unique()
   
   # 날짜 기준 데이터프레임 생성
   date_df = pd.DataFrame({'날짜': all_dates})
   date_df = date_df.sort_values('날짜')
   
   # 데이터 병합
   dfs = [
		date_df, 
		economic, 
		trends,
		weather
   ]
   
   result = dfs[0]
   for df in dfs[1:]:
       result = pd.merge(result, df, on='날짜', how='left')
   
   # 결과 저장
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result


In [ ]:
result = merge_all_data()

### 결측치 채우기

 "filled_merged_all_data.csv" 생성

In [2]:
import pandas as pd

def fill_missing_values(file_path, output_path):
    """
    CSV 파일에서 결측치를 `ffill`을 사용하여 채우되, 이전 값이 없으면 그대로 둠.
    
    Args:
        file_path (str): 입력 CSV 파일 경로.
        output_path (str): 처리된 데이터를 저장할 경로.
    
    Returns:
        pd.DataFrame: 결측치가 채워진 데이터프레임.
    """
    
    # CSV 파일 로드
    df = pd.read_csv(file_path)

    # 결측치 확인 (처리 전)
    missing_before = df.isnull().sum()

    # 결측치 `ffill`로 채우기 (이전 값이 없으면 그대로 둠)
    df.fillna(method='ffill', inplace=True)

    # 결측치 확인 (처리 후)
    missing_after = df.isnull().sum()

    # 변경된 결측치 개수 출력
    missing_summary = pd.DataFrame({'Before': missing_before, 'After': missing_after})
    print("\n🔍 결측치 처리 전후 비교:")
    print(missing_summary)

    # 처리된 데이터 저장
    df.to_csv(output_path, index=False)
    print(f"\n✅ 처리된 데이터가 '{output_path}'에 저장되었습니다.")

    return df




In [3]:

# ✅ 실행 예시
input_file = "../../data/features/merged_all_data.csv"  # 원본 데이터 경로
output_file = "filled_merged_all_data.csv"  # 저장될 파일 경로

# 함수 실행
filled_df = fill_missing_values(input_file, output_file)


🔍 결측치 처리 전후 비교:
                Before  After
날짜                   0      0
활연어_거래량(톤)        1107   1097
활연어_가격(NOK/kg)    1107   1097
광어_대              1938   1098
광어_중              1938   1098
...                ...    ...
우럭_파주기_22189       487    355
참돔_기온_22190        452    342
참돔_수온_22107        203      0
참돔_습도_22190        454    342
참돔_파주기_22190       414    342

[122 rows x 2 columns]

✅ 처리된 데이터가 'filled_merged_all_data.csv'에 저장되었습니다.


C:\Users\user\AppData\Local\Temp\ipykernel_38656\1915651589.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


### 어종별 가격 데이터 파일 분리


In [ ]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('pp/item_price_lag_filled.csv')

# 유니크한 item 목록 확인
unique_items = df['item'].unique()

# 각 item별로 데이터 분리하여 저장
for item in unique_items:
    # 해당 item의 데이터만 필터링
    item_df = df[df['item'] == item]
    
    # 파일명에 사용할 수 있게 item 이름 처리 (특수문자 등 제거)
    file_name = f"{item.replace(' ', '_').replace('/', '_')}_price_data.csv"
    
    # CSV 파일로 저장
    item_df.to_csv(file_name, index=False, encoding='utf-8')
    
    # 저장 완료 메시지 출력
    print(f"{item} 데이터가 {file_name}으로 저장되었습니다.")
    print(f"행 수: {len(item_df)}")

# 전체 처리 완료 메시지
print("\n전체 품목 분리 저장이 완료되었습니다.")
print(f"총 {len(unique_items)}개의 파일이 생성되었습니다.")

### 변수랑 인어교주해적단 가격이랑 합치기

### 합친 데이터에 타임래그 적용하기 